In [1]:
from tensorflow.python.client import device_lib

def get_available_devices():  
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())  

['/cpu:0']


In [2]:
# Load pickled data
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os

csv_filename = 'driving_log.csv'
data_path = "../sim_data/data/"
test_data_path = "../sim_data/test_data/"

data_csv_path = data_path + csv_filename
test_data_csv_path = test_data_path + csv_filename

test_data_csv_df = pd.read_csv(test_data_csv_path, index_col=False)
test_data_csv_df.columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

data_test = test_data_csv_df.sample(n=len(test_data_csv_df))

data_csv_df = pd.read_csv(data_csv_path, index_col=False)
data_csv_df.columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
data = data_csv_df.sample(n=len(data_csv_df))

split_train = int(0.7*len(data))
split_valid = int(0.9*len(data))
split_test = len(data)

data_train = data[:split_train]
data_valid = data[split_train:split_valid]
data_test = data[split_valid:split_test]

print(len(data_train))
print(len(data_valid))
print(len(data_test))

img_paths = data_train['center'][512:1024].values
print(img_paths[0])

29830
8523
4262
/home/boliu1/Projects/CND/sim_data/data/IMG/center_2017_10_16_01_53_30_027.jpg


In [3]:
import cv2
from PIL import Image
from keras.backend import tf as ktf

def read_img(path):
    """
    Returns a numpy array image from path
    """
    img = cv2.imread(path.strip())
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_img(path):
    image = Image.open(path)
    image = image.convert('RGB')
    return image

def preprocess_data(data):
    """
    Return processed data
    """
    return data['center'], data['steering']
    
def images_to_nparray(image_list, w=160, h=320):
    output = []
    for image in image_list:
        img = np.array(list(image.getdata()), dtype='uint8')
        img = np.reshape(img, (w, h, 3))
        output.append(img)
    
    return np.array(output, dtype='uint8')

def get_batch(data, batch_size=512):
    batch = data.sample(batch_size)
    img_paths = batch['center'].values
    steerings = batch['steering'].values
    
    img_list = []
    for path in img_paths:
        img = load_img(path)
        img = np.asarray(img, dtype='float32')
        img_list.append(img)
    
    imgs = np.array(img_list, dtype='float32')
    strs = np.array(steerings, dtype='float32')
    
    return imgs, strs
    
    
def train_sample_generator(train_df, batch_size=512):
    """
    Generate a batch of training data
    """
    while True:
        yield get_batch(train_df, batch_size=batch_size)
        
        
def valid_sample_generator(valid_df, batch_size_valid=512):
    return train_sample_generator(valid_df, batch_size_valid)

Using TensorFlow backend.


In [ ]:
from keras.models import Sequential
from keras.layers import *
from keras.models import Sequential
from keras.backend import tf as ktf
from keras.optimizers import Adam

def crop_imgs(imgs, top=60, bottom=140):
    """
    Returns croppped image tensor
    """
    return imgs[:,top:bottom,:,:]

def nvidia_net(input_shape=(160, 320, 3)):    
    model = Sequential()
    
    # 160x320x3 -> 80x320x3
    model.add(Lambda(crop_imgs, input_shape=input_shape))
    
    # 80x320x3 -> 80x320x3
    model.add(Lambda(lambda x: x/255.-0.5))
    
    # -> 38x158x24
    model.add(Conv2D(24, kernel_size=(5,5),
                     strides=(2,2),
                     activation='relu'))
    
    # -> 17x77x36
    model.add(Conv2D(36, kernel_size=(5,5),
                     strides=(2,2),
                     activation="relu"))
    
    # -> 7x37x48
    model.add(Conv2D(48, kernel_size=(5, 5), 
                     strides=(2,2),
                     activation="relu"))
    
    # -> 3x18x64
    model.add(Conv2D(64, kernel_size=(3, 3),
                     strides=(2,2),
                     activation="relu"))

    # -> 1x16x64
    model.add(Conv2D(64, kernel_size=(3, 3), 
                     activation="relu"))
    
    model.add(Flatten())
    model.add(Dense(1164, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(50, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation="relu"))

    model.add(Dense(1))
    
    model.compile(optimizer=Adam(lr=0.001), loss='mse')
    return model


In [ ]:
BATCH_SIZE = 512
BATCH_SIZE_VALID = 128
EPOCHS = 5

DATA_PATH = "../sim_data/data/"


img = read_img(img_paths[0])
input_shape = img.shape
model = nvidia_net(input_shape)

n_data_train = len(data_train)
n_steps = int(n_data_train/BATCH_SIZE)

n_data_valid = len(data_valid)
n_steps_valid = int(n_data_valid/BATCH_SIZE_VALID)

values = model.fit_generator(train_sample_generator(data_train, BATCH_SIZE),
                             validation_data=valid_sample_generator(data_valid, BATCH_SIZE_VALID),
                             steps_per_epoch=n_steps,
                             validation_steps=n_steps_valid,
                             epochs=EPOCHS)

model.save('nvidia_net.h5')

Epoch 1/5
58/58 [==============================] - 210s - loss: 0.0152 - val_loss: 0.0129
Epoch 2/5
58/58 [==============================] - 210s - loss: 0.0078 - val_loss: 0.0040
Epoch 3/5
58/58 [==============================] - 210s - loss: 0.0050 - val_loss: 0.0036
Epoch 4/5
58/58 [==============================] - 211s - loss: 0.0043 - val_loss: 0.0036
Epoch 5/5
57/58 [============================>.] - ETA: 3s - loss: 0.0038